In [95]:
import requests
import json
import os
import pandas as pd
import time

In [97]:
# setting output directory

output_path = "/data/dssg/occrp/data/aleph-exploration/random_sample_aleph"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [98]:
input_path = "/home/jsanchez/dssgxdfki2022-occrp/notebooks/aleph_exploration/uniform_random_sample_aleph"

collections_file = open(os.path.join(input_path, "1000-random-public-documents.json"))
type(collections_file)
document_data = json.load(collections_file)
# with open(os.path.join(input_path, "1000-random-public-documents.json")) as collections_file:
#    document_data = json.load(collections_file)["took"]

# collections_df = pd.DataFrame(document_data["results"])

In [99]:
document_df = pd.DataFrame(document_data["hits"]["hits"])
document_df["collection_id"] = [i["collection_id"] for i in document_df["_source"]]
document_df["collection_id_str"] = [str(i) for i in document_df["collection_id"]]

api_init = "https://aleph.occrp.org/api/2/entities/"

document_df["api_url"] = [api_init + i for i in document_df["_id"]]
print(document_df.columns)
print(document_df.head)

Index(['_index', '_type', '_id', '_score', '_source', 'collection_id',
       'collection_id_str', 'api_url'],
      dtype='object')
<bound method NDFrame.head of                     _index _type  \
0    aleph-entity-pages-v2  _doc   
1    aleph-entity-pages-v2  _doc   
2    aleph-entity-pages-v2  _doc   
3    aleph-entity-pages-v2  _doc   
4    aleph-entity-pages-v2  _doc   
..                     ...   ...   
995  aleph-entity-pages-v2  _doc   
996  aleph-entity-pages-v2  _doc   
997  aleph-entity-pages-v2  _doc   
998  aleph-entity-pages-v2  _doc   
999  aleph-entity-pages-v2  _doc   

                                                   _id    _score  \
0     3489219.1727f29cc0da0fb035269647c6ca15855913fce2  1.000000   
1    16058971.d5853359f456269165ea5c8142a7e73d15f27c94  1.000000   
2    50277102.1949be476bf51be8034a6f9b7d900847032621fe  1.000000   
3     6840256.fe40d0d686a14c357da88bdc6792c0ed47379b85  0.999999   
4     2992121.5ea63dd2af8b70242a27b78bab6ad6db5d0c4d28  0.999999

In [100]:
document_df["api_url"].iloc[0]

'https://aleph.occrp.org/api/2/entities/3489219.1727f29cc0da0fb035269647c6ca15855913fce2'

In [104]:
## Obtaining document types before downloading all:
mimeType = []

for i, row in document_df.iterrows():
    document_request = requests.get(row["api_url"]).json()
    document_request_pd = pd.DataFrame.from_dict(document_request, orient="index")

    try:
        mimeType.append(document_request_pd[0]["properties"]["mimeType"][0])
    except:
        mimeType.append("")

    time.sleep(0.01)

    # if mimeType in ["",","","",""]:
    # with open(
    #    os.path.join(output_path, f"{row['_id']}"), "wb"
    # ) as document_file:
    #    document_response = requests.get(document_request["links"]["file"])
    #    document_file.write(document_response.content)

document_df["mimeType"] = mimeType

In [105]:
document_df["mimeType"].value_counts()

application/pdf                                                            821
image/tiff                                                                 114
application/vnd.ms-word                                                     50
application/vnd.openxmlformats-officedocument.wordprocessingml.document     12
text/rtf                                                                     1
application/vnd.ms-powerpoint                                                1
                                                                             1
Name: mimeType, dtype: int64

In [108]:
document_reduced_df = document_df[[(i in ["application/pdf", "image/tiff"]) for i in document_df["mimeType"]]]
print(document_reduced_df.shape)

(935, 9)


In [109]:
for i, row in document_reduced_df.iterrows():
    document_request = requests.get(row["api_url"]).json()
    document_request_pd = pd.DataFrame.from_dict(document_request, orient="index")

    ext = None
    if row["mimeType"] == "application/pdf":
        ext = ".pdf"
    if row["mimeType"] == "image/tiff":
        ext = ".tiff"

    with open(os.path.join(output_path, f"{row['_id']}" + ext), "wb") as document_file:
        pdf_response = requests.get(document_request["links"]["file"])
        document_file.write(pdf_response.content)